In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.layers import concatenate, Dropout, GlobalAveragePooling2D
from tensorflow.keras.layers import BatchNormalization, Activation, AveragePooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau

# seed
import os
seed = 123
os.environ['PYTHONHASHSEED'] = str(seed)
np.random.seed(seed)
tf.set_random_seed(seed)

In [2]:
train = np.load('data/train.npy', allow_pickle = 'True')
test = np.load('data/test.npy', allow_pickle = 'True')

In [3]:
x = train[:,2:]
x = np.reshape(x, (-1, 28, 28, 1))

x_letter = train[:,1]
x_letter = np.reshape(x_letter, (-1, 1))
en = OneHotEncoder()
x_letter = en.fit_transform(x_letter).toarray()

y = train[:,0]
y = np.reshape(y, (-1, 1))
en = OneHotEncoder()
y = en.fit_transform(y).toarray()

print(x.shape)
print(x_letter.shape)
print(y.shape)

(2048, 28, 28, 1)
(2048, 26)
(2048, 10)


In [4]:
valid_size = 48
valid_x = x[-valid_size:]
x = x[:-48]

valid_x_letter = x_letter[-valid_size:]
x_letter = x_letter[:-48]

valid_y = y[-valid_size:]
y = y[:-48]

print(x.shape)
print(valid_x.shape)
print(x_letter.shape)
print(valid_x_letter.shape)
print(y.shape)
print(valid_y.shape)

(2000, 28, 28, 1)
(48, 28, 28, 1)
(2000, 26)
(48, 26)
(2000, 10)
(48, 10)


In [5]:
image_generator = ImageDataGenerator(width_shift_range=0.1,
                                     height_shift_range=0.1, 
                                     zoom_range=[0.8,1.2],
                                     shear_range=10)

In [6]:
x_total = x.copy()
def augment(x):
    aug_list = []
    for i in range(x.shape[0]):
        num_aug = 0
        tmp = x[i]
        tmp = tmp.reshape((1,) + tmp.shape)
        for x_aug in image_generator.flow(tmp, batch_size = 1) :
            if num_aug >= 1:
                break
            aug_list.append(x_aug[0])
            num_aug += 1
    aug_list = np.array(aug_list)
    return aug_list

n = 4
for i in range(n):
    arr = augment(x)
    x_total = np.concatenate((x_total, arr), axis=0)
    if i > n:
        break

print(x_total.shape)

(10000, 28, 28, 1)


In [7]:
y_total = y.copy()
for i in range(n):
    arr = y.copy()
    y_total = np.concatenate((y_total, arr), axis=0)

print(y_total.shape)

(10000, 10)


In [8]:
x_letter_total = x_letter.copy()
for i in range(n):
    arr = x_letter.copy()
    x_letter_total = np.concatenate((x_letter_total, arr), axis=0)
    
print(x_letter_total.shape)

(10000, 26)


In [9]:
x_train, x_val, y_train, y_val = train_test_split(x_total, y_total, test_size=0.2, shuffle=True)#, stratify=y_total)
x_letter_train = x_letter_total[:x_train.shape[0],:]
x_letter_val = x_letter_total[x_train.shape[0]:,:]

print(x_train.shape)
print(x_val.shape)
print(y_train.shape)
print(y_val.shape)
print(x_letter_train.shape)
print(x_letter_val.shape)

(8000, 28, 28, 1)
(2000, 28, 28, 1)
(8000, 10)
(2000, 10)
(8000, 26)
(2000, 26)


In [10]:
def Conv_block(x, growth_rate, activation='relu'):
    x_l = BatchNormalization()(x)
    x_l = Activation(activation)(x_l)
    x_l = Conv2D(growth_rate*4, (1,1), padding='same', kernel_initializer='he_normal')(x_l)
    
    x_l = BatchNormalization()(x_l)
    x_l = Activation(activation)(x_l)
    x_l = Conv2D(growth_rate, (3,3), padding='same', kernel_initializer='he_normal')(x_l)
    
    x = concatenate([x, x_l])
    return x

def Dense_block(x, layers, growth_rate=32):
    for i in range(layers):
        x = Conv_block(x, growth_rate)
    return x

def Transition_layer(x, compression_factor=0.5, activation='relu'):
    reduced_filters = int(tf.keras.backend.int_shape(x)[-1] * compression_factor)
    
    x = BatchNormalization()(x)
    x = Activation(activation)(x)
    x = Conv2D(reduced_filters, (1,1), padding='same', kernel_initializer='he_normal')(x)
    
    x = AveragePooling2D((2,2), padding='same', strides=2)(x)
    return x

def DenseNet(model_input, classes, densenet_type='DenseNet-121'):
    x = Conv2D(base_growth_rate*2, (5,5), padding='same', strides=1,
               kernel_initializer='he_normal')(model_input)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    
    x = MaxPooling2D((2,2), padding='same', strides=1)(x)
    
    x = Dense_block(x, layers_in_block[densenet_type][0], base_growth_rate)
    x = Transition_layer(x, compression_factor=0.5)
    x = Dense_block(x, layers_in_block[densenet_type][1], base_growth_rate)
    x = Transition_layer(x, compression_factor=0.5)
    x = Dense_block(x, layers_in_block[densenet_type][2], base_growth_rate)
    #x = Transition_layer(x, compression_factor=0.5)
    #x = Dense_block(x, layers_in_block[densenet_type][3], base_growth_rate)
    
    x = GlobalAveragePooling2D()(x)
    
    model_output = Dense(classes, activation='softmax', kernel_initializer='he_normal')(x)
    
    model = Model(model_input, model_output, name=densenet_type)
    
    return model

In [11]:
layers_in_block = {'DenseNet-121':[6, 12, 24, 16],
                   'DenseNet-169':[6, 12, 32, 32],
                   'DenseNet-201':[6, 12, 48, 32],
                   'DenseNet-265':[6, 12, 64, 48],
                   'myDenseNet':[8, 12, 16, 32]}

base_growth_rate = 32

model_input = Input(shape=(28,28,1))
classes = 10

model = DenseNet(model_input, classes, 'DenseNet-121')

model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 28, 28, 1)    0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 28, 28, 64)   1664        input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization (BatchNorma (None, 28, 28, 64)   256         conv2d[0][0]                     
__________________________________________________________________________________________________
activation (Activation)         (None, 28, 28, 64)   0           batch_normalization[0][0]        
__________________________________________________________________________________________________
max_poolin

In [12]:
model.compile(optimizer = 'adam', metrics = ['accuracy'], loss = 'categorical_crossentropy')

es = EarlyStopping(monitor='val_loss', patience=10, mode='min', verbose=1)
cp = ModelCheckpoint('./models/{epoch:02d}-{val_acc:.4f}.h5', monitor='val_loss',
                     save_best_only=True, mode='min')
lr = ReduceLROnPlateau(monitor='val_loss', patience=5, verbose=1, factor=0.5, min_lr=0.00001)

history = model.fit(x_train, y_train,
                    validation_data=(x_val, y_val), 
                    batch_size=64, epochs=100, verbose=1, callbacks = [cp, lr])

Train on 8000 samples, validate on 2000 samples
Epoch 1/100
8000/8000 [==============================] - 75s 9ms/step - loss: 1.8376 - acc: 0.3841 - val_loss: 6.4279 - val_acc: 0.1750
Epoch 2/100
8000/8000 [==============================] - 48s 6ms/step - loss: 0.9368 - acc: 0.6733 - val_loss: 1.6605 - val_acc: 0.5450
Epoch 3/100
8000/8000 [==============================] - 48s 6ms/step - loss: 0.6251 - acc: 0.7854 - val_loss: 1.3610 - val_acc: 0.6600
Epoch 4/100
8000/8000 [==============================] - 48s 6ms/step - loss: 0.4453 - acc: 0.8461 - val_loss: 1.0267 - val_acc: 0.6850
Epoch 5/100
8000/8000 [==============================] - 48s 6ms/step - loss: 0.3570 - acc: 0.8786 - val_loss: 0.7759 - val_acc: 0.7670
Epoch 6/100
8000/8000 [==============================] - 48s 6ms/step - loss: 0.2713 - acc: 0.9099 - val_loss: 0.7378 - val_acc: 0.7995
Epoch 7/100
8000/8000 [==============================] - 48s 6ms/step - loss: 0.2099 - acc: 0.9270 - val_loss: 1.7732 - val_acc: 0.6330


Epoch 57/100
8000/8000 [==============================] - 48s 6ms/step - loss: 1.3277e-04 - acc: 1.0000 - val_loss: 0.0503 - val_acc: 0.9885
Epoch 58/100
8000/8000 [==============================] - 48s 6ms/step - loss: 1.2736e-04 - acc: 1.0000 - val_loss: 0.0506 - val_acc: 0.9885
Epoch 59/100
8000/8000 [==============================] - 48s 6ms/step - loss: 1.2728e-04 - acc: 1.0000 - val_loss: 0.0509 - val_acc: 0.9885

Epoch 00059: ReduceLROnPlateau reducing learning rate to 1.5625000742147677e-05.
Epoch 60/100
8000/8000 [==============================] - 48s 6ms/step - loss: 1.1105e-04 - acc: 1.0000 - val_loss: 0.0505 - val_acc: 0.9890
Epoch 61/100
8000/8000 [==============================] - 48s 6ms/step - loss: 9.4993e-05 - acc: 1.0000 - val_loss: 0.0501 - val_acc: 0.9885
Epoch 62/100
8000/8000 [==============================] - 48s 6ms/step - loss: 1.3793e-04 - acc: 1.0000 - val_loss: 0.0505 - val_acc: 0.9890
Epoch 63/100
8000/8000 [==============================] - 48s 6ms/step -

In [16]:
from tensorflow.keras.models import load_model
best_model = load_model('models/DenseNet121_LR_aug4.h5')

In [17]:
evaluate = best_model.evaluate(valid_x, valid_y)
evaluate

48/48 [==============================] - 7s 138ms/step


[0.6857667565345764, 0.875]

In [15]:
#submission = pd.read_csv('data/val.csv')
#submission['digit'] = np.argmax(best_model.predict([x_test, x_letter_test]), axis=1)
#submission.to_csv('data/val(CNN).csv', index=False)